In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime

# RFV


df_compras = pd.read_csv('./input/dados.csv',
                         infer_datetime_format=True,
                         parse_dates=['DiaCompra'])
df_compras.head(20)

df_compras.shape

df_compras['DiaCompra'].min()

df_compras['DiaCompra'].max()



dia_atual = datetime(2021, 12, 9)
print(dia_atual)

#group by customers and check last date of purshace
df_recencia = df_compras.groupby(by='ID_cliente',
                                 as_index=False)['DiaCompra'].max()
df_recencia.columns = ['ID_cliente', 'DiaUltimaCompra']
df_recencia.head()

df_recencia['Recencia'] = df_recencia['DiaUltimaCompra'].apply(
    lambda x: (dia_atual - x).days)

df_compras['DiaCompra'].max()

dia_atual = datetime(2021,12,9)
print(dia_atual)

df_recencia.head()

df_recencia.drop('DiaUltimaCompra', axis=1, inplace=True)

## Frequência

**Quantas vezes cada cliente comprou com a gente?**

df_compras.head(15)

df_frequencia = df_compras[['ID_cliente', 'CodigoCompra'
                            ]].groupby('ID_cliente').count().reset_index()
df_frequencia.columns = ['ID_cliente', 'Frequencia']
df_frequencia.head()

## Valor


df_valor = df_compras[['ID_cliente', 'ValorTotal'
                       ]].groupby('ID_cliente').sum().reset_index()
df_valor.columns = ['ID_cliente', 'Valor']
df_valor.head()

## Criando a tabela RFV

df_RF = df_recencia.merge(df_frequencia, on='ID_cliente')
df_RF.head()

df_RFV = df_RF.merge(df_valor, on='ID_cliente')
df_RFV.set_index('ID_cliente', inplace=True)
df_RFV.head()

## Segmentação de clientes utilizando o RFV

=
### Quartis para o RFV

quartis = df_RFV.quantile(q=[0.25, 0.5, 0.75])
quartis

quartis.to_dict()

### Criando os segmentos

def recencia_class(x, r, q_dict):
    """Classifica como melhor o menor quartil 
       x = valor da linha,
       r = recencia,
       q_dict = quartil dicionario   
    """
    if x <= q_dict[r][0.25]:
        return 'A'
    elif x <= q_dict[r][0.50]:
        return 'B'
    elif x <= q_dict[r][0.75]:
        return 'C'
    else:
        return 'D'


def freq_val_class(x, fv, q_dict):
    """Classifica como melhor o maior quartil 
       x = valor da linha,
       fv = frequencia ou valor,
       q_dict = quartil dicionario   
    """
    if x <= q_dict[fv][0.25]:
        return 'D'
    elif x <= q_dict[fv][0.50]:
        return 'C'
    elif x <= q_dict[fv][0.75]:
        return 'B'
    else:
        return 'A'

df_RFV['R_quartil'] = df_RFV['Recencia'].apply(recencia_class,
                                                args=('Recencia', quartis))
df_RFV['F_quartil'] = df_RFV['Frequencia'].apply(freq_val_class,
                                                  args=('Frequencia', quartis))
df_RFV['V_quartil'] = df_RFV['Valor'].apply(freq_val_class,
                                             args=('Valor', quartis))

df_RFV.head()

df_RFV['RFV_Score'] = (df_RFV.R_quartil + df_RFV.F_quartil +
                       df_RFV.V_quartil)
df_RFV.head()

df_RFV['RFV_Score'].value_counts()

quartis



df_RFV[df_RFV['RFV_Score'] == 'AAA'].sort_values('Valor',
                                                 ascending=False).head(10)

### Ações de marketing/CRM

dict_acoes = {
    'AAA':
    'Enviar cupons de desconto, Pedir para indicar nosso produto pra algum amigo, Ao lançar um novo produto enviar amostras grátis pra esses.',
    'DDD':
    'Churn! clientes que gastaram bem pouco e fizeram poucas compras, fazer nada',
    'DAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar',
    'CAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar'
}

df_RFV['acoes de marketing/crm'] = df_RFV['RFV_Score'].map(dict_acoes)

df_RFV.head()

df_RFV.to_excel('./output/RFV.xlsx')



